In [ ]:
import os
import gc
import glob
import numpy as np
import pandas as pd

import uproot
import trimesh
from trimesh.points import PointCloud
from scipy.constants import epsilon_0, e as q_e

import matplotlib.pyplot as plt
from matplotlib.colors import Normalize
from matplotlib.colorbar import ColorbarBase
from matplotlib.colors import LogNorm
import matplotlib.colors as mcolors
import matplotlib as mpl
# Enable LaTeX rendering
mpl.rcParams['text.usetex'] = False
# Set the global font size
mpl.rcParams.update({'font.size': 12})

from common_functions import *

# from scipy.spatial import cKDTree
# import pyvista as pv
# pv.set_jupyter_backend('trame')  # or 'panel' if using panel

In [ ]:
# import geometry
geometry = trimesh.load_mesh('../sphere-charging/geometry/isolated_grains_interpolated.stl') 

# Import ROOT files & calculate PE yield 
 
runs with 10 million particles, entire world as sensitive region, monoenergetic UV source

In [ ]:
# -----------------------------
# User configuration
# -----------------------------
CONFIG = ["onlyUV"]  # Can be ["onlysolarwind","onlyphotoemission","allparticles", etc.]
DIRECTORY = "../build-realistic-grains/root/"
SELECT_NUM = 10_000_000
FILELIST = sorted(glob.glob(f"{DIRECTORY}/*iteration0*{CONFIG}*5.3eV*_num{SELECT_NUM}.root"))

dataframes = {}  # Store summary dataframes per config
percent_emission,photon_energy = [],[]

# -----------------------------
# Utility functions
# -----------------------------
def parse_filename(fname):
    """Extract iteration number and config from filename."""
    parts = fname.split("_")
    iteration = int("".join(filter(str.isdigit, parts[1])))
    config = parts[2]
    return iteration, config

def compute_differential(counts, bin_edges):
    """Convert counts to differential flux (counts per bin width)."""
    widths = np.diff(bin_edges)
    return counts / widths, widths

# Keep only necessary columns to save memory
needed_cols = ["Event_Number", "Particle_Type", "Parent_ID",
                "Volume_Name_Pre","Volume_Name_Post",
                "Kinetic_Energy_Pre_MeV", "Kinetic_Energy_Post_MeV",
                "Pre_Step_Position_mm", "Post_Step_Position_mm"]

# -----------------------------
# Main loop over ROOT files
# -----------------------------
for filepath in FILELIST:

    filename = os.path.basename(filepath)
    print(f"Processing: {filename}")

    iteration, cfg = parse_filename(filename)

    # -----------------------------
    # Read ROOT file
    # -----------------------------
    df = read_rootfile(filename, directory_path=DIRECTORY) #, columns=needed_cols)

    # -----------------------------
    # Select particles (avoid repeated filtering)
    # -----------------------------
    df_e = df[df["Particle_Type"] == "e-"]
    df_gamma = df[df["Particle_Type"] == "gamma"]

    if len(df_e) > 0:

        # First photoelectron (creation)
        photoPE = df_e.drop_duplicates(subset="Event_Number", keep="first")

        # Incident gammas
        incident_gamma = df_gamma[df_gamma["Parent_ID"] == 0].drop_duplicates(subset="Event_Number", keep="first")

        # Last electron step
        last_e = df_e.drop_duplicates(subset="Event_Number", keep="last")

        # Boolean mask for electrons reaching top surface
        vol_post = last_e["Volume_Name_Post"].to_numpy()
        vol_pre = last_e["Volume_Name_Pre"].to_numpy()
        top_mask = np.logical_or(vol_post == "physical_cyclic", vol_pre == "physical_cyclic")
        top_e = last_e[top_mask]

        # -----------------------------
        # Post-step positions
        # -----------------------------
        post_pos = np.vstack(top_e["Post_Step_Position_mm"])
        x_post, z_post = post_pos[:, 0], post_pos[:, 2]

        # Identify electrons near top/bottom edges (99% rule)
        Zmin, Zmax = z_post.min(), z_post.max()
        edge_mask = (z_post >= Zmax * 0.99) | (z_post <= Zmin * 0.99)
        edge_indices = np.nonzero(edge_mask)[0]

        # # -----------------------------
        # # Plot positions (XZ plane)
        # # -----------------------------
        # plt.figure(figsize=(5, 4))
        # plt.plot(x_post, z_post, "k.", ms=0.1)
        # plt.plot(x_post[edge_indices], z_post[edge_indices], "b.", ms=0.2)
        # plt.title(f"Final e- positions (iteration {iteration})")
        # plt.xlabel("X [mm]")
        # plt.ylabel("Z [mm]")
        # plt.show()

        # -----------------------------
        # Histogram bins
        # -----------------------------
        #bins = np.logspace(-4, 3.2, 200)
        bins = np.linspace(0,10,50)

        # Convert energies to NumPy arrays (eV)
        E_all = photoPE["Kinetic_Energy_Pre_MeV"].to_numpy(dtype=np.float64) * 1e6
        E_gamma = incident_gamma["Kinetic_Energy_Pre_MeV"].to_numpy(dtype=np.float64) * 1e6
        E_emit = np.vstack(top_e["Kinetic_Energy_Post_MeV"]).astype(np.float64).ravel()[edge_mask] * 1e6

        # -----------------------------
        # Compute histograms
        # -----------------------------
        hist_all, bin_edges = np.histogram(E_all, bins=bins)
        hist_emit, _ = np.histogram(E_emit, bins=bins)
        hist_gamma, _ = np.histogram(E_gamma, bins=bins)

        diff_all, widths = compute_differential(hist_all, bin_edges)
        diff_emit, _ = compute_differential(hist_emit, bin_edges)

        bin_centers = 0.5 * (bin_edges[1:] + bin_edges[:-1])

        # Mask small values for log-scale plotting
        # threshold = 0.1
        # diff_all_mask = np.where(diff_all > threshold, diff_all, np.nan)
        # diff_emit_mask = np.where(diff_emit > threshold, diff_emit, np.nan)

        # -----------------------------
        # Plot differential flux
        # -----------------------------
        fig, ax = plt.subplots(figsize=(5, 3.2))
        #ax.step(bin_centers, diff_all_mask, where='mid', color='navy', lw=1.5, label='All photoelectrons')
        #ax.fill_between(bin_centers, diff_all_mask, step='mid', color='navy', alpha=0.3)
        ax.step(bin_centers, diff_emit, where='mid', color='darkgreen', lw=1.5, label='Emitted e-')
        ax.fill_between(bin_centers, diff_emit, step='mid', color='darkgreen', alpha=0.3)
        #ax.set_xscale("log")
        #ax.set_yscale("log")
        ax.set_xlim([bins[0], bins[-1]])
        ax.set_ylim(bottom=10)
        ax.set_xlabel("Photoelectron Energy (eV)")
        ax.set_ylabel("Differential Flux")
        ax.legend()

        # -----------------------------
        # Print statistics
        # -----------------------------
        print(f"Average PE energy: {E_all.mean():.2f} eV")
        print(f"# photoelectrons: {len(photoPE)} ({len(photoPE)/len(incident_gamma)*100:.3f}% of incident)")
        print(f"# emitted e-:    {len(edge_indices)} ({len(edge_indices)/len(photoPE)*100:.3f}% of absorbed)")
        print(f"Emission per photon: {len(edge_indices)/len(incident_gamma)*100:.3f}%")

        plt.show()

        # -----------------------------
        # Store dataframes and emission
        # -----------------------------
        dataframes[cfg] = dict(last_e=last_e, photoPE=photoPE, incident=incident_gamma)
        percent_emission.append(len(edge_indices) / len(incident_gamma))
        photon_energy.append(np.mean(incident_gamma["Kinetic_Energy_Pre_MeV"]))

        # -----------------------------
        # Clean up large variables
        # -----------------------------
        #del df, df_e, df_gamma, photoPE, last_e, top_e, post_pos, E_all, E_gamma, E_emit
        #del hist_all, hist_emit, hist_gamma, diff_all, diff_emit, diff_all_mask, diff_emit_mask
        gc.collect()
        print("-"*72)
    else:
        print("NO photoelectrons created...")
        print("-"*72)

In [ ]:
12-max(E_emit)

In [ ]:
fig, ax = plt.subplots(figsize=(5, 3.2))
#ax.step(bin_centers, diff_all_mask, where='mid', color='navy', lw=1.5, label='All photoelectrons')
#ax.fill_between(bin_centers, diff_all_mask, step='mid', color='navy', alpha=0.3)
ax.step(bin_centers, diff_emit, where='mid', color='darkgreen', lw=1.5, label='Emitted e-')
ax.fill_between(bin_centers, diff_emit, step='mid', color='darkgreen', alpha=0.3)
#ax.set_xscale("log")
ax.set_yscale("log")
ax.set_xlim([bins[0], bins[-1]])
ax.set_ylim(bottom=10)
ax.set_xlabel("Photoelectron Energy (eV)")
ax.set_ylabel("Differential Flux")
ax.legend()

In [ ]:
G4EmStandardPhysicsWVI: UV photons = 10.2 eV (500,000 particles)

- with lowest energy e+- = 0 eV, (/run/setCut 10 um, /cuts/setLowEdge 10 eV)

- with lowest energy e+- = 0 eV, (/run/setCut 10 um)
Average PE energy: 2.05 eV
# photoelectrons: 198388 (39.678% of incident)
# emitted e-:    714 (0.360% of absorbed)
Emission per photon: 0.143%

G4EmStandardPhysicsWVI: UV photons = 10 eV (100,000 particles)

- with lowest energy e+- = 0 eV, (/run/setCut 0.01 nm, /cuts/setLowEdge 1 eV)
-> took forever ... didn't get past event0

- with only lowest energy e+- = 0 eV (100,000 particles)
Average PE energy: 1.85 eV
# photoelectrons: 41254 (41.254% of incident)
# emitted e-:    142 (0.344% of absorbed)
Emission per photon: 0.142%

G4EmStandardPhysics_option4: UV photons = 10 eV (100,000 particles)

- with lowest energy e+- = 0 eV, (/run/setCut  0.01 nm, /cuts/setLowEdge 1 eV)
Average PE energy: 1.85 eV
# photoelectrons: 41038 (41.038% of incident)
# emitted e-:    139 (0.339% of absorbed)
Emission per photon: 0.139%

-with lowest energy e+- = 10 eV
Average PE energy: 1.85 eV
# photoelectrons: 41037 (41.037% of incident)
# emitted e-:    85 (0.207% of absorbed)
Emission per photon: 0.085%
-> no continous decrease (all energy of e- lost in one step)

In [ ]:
plt.plot(df_e["Kinetic_Energy_Diff_eV"], '.')

In [ ]:
df_e

In [ ]:
incident_gamma = df_gamma[df_gamma["Parent_ID"] == 0].drop_duplicates(subset="Event_Number", keep="first")

# Plot dependence with energy & fit to extrapolate work function